In [1]:
!pip install opencv-python boto3 botocore ffmpeg-python moviepy --upgrade -q

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [ ]:
%%writefile rekognition.py

import boto3
import botocore
import time

def boto3_client():
    return boto3.client('rekognition')

def check_format_and_size(filename, size):
    if filename.split('.')[-1] in ['mp4', 'mov']:
        if size < 10*1024*1024*1024:
            return True
    return False

def start_face_detection(bucket, video, size, reko_client=None):
    assert check_format_and_size(video, size)
    if reko_client == None:
        reko_client = boto3.client('rekognition')
    response = reko_client.start_face_detection(Video={'S3Object': {'Bucket': bucket, 'Name': video}})
    return response['JobId']

def wait_for_completion(job_id, wait_time_in_s=30, reko_client=None):
    if reko_client == None:
        reko_client = boto3.client('rekognition')
    response = reko_client.get_face_detection(JobId=job_id)
    while (response['JobStatus'] == 'IN_PROGRESS'):
        print('.', end='')
        time.sleep(wait_time_in_s)
        response = reko_client.get_face_detection(JobId=job_id)
    print('Complete')
    return response

def get_timestamps_and_faces(response, job_id, reko_client=None):
    final_timestamps = {}
    next_token = "Y"
    first_round = True
    while next_token != "":
        print('.', end='')
        # Set some variables if it's the first iteration
        if first_round:
            next_token = ""
            first_round = False
        # Query Reko Video
        response = reko_client.get_face_detection(JobId=job_id, NextToken=next_token)
        # Iterate over every face
        for face in response['Faces']:
            f = face["Face"]["BoundingBox"]
            t = str(face["Timestamp"])
            time_faces = final_timestamps.get(t)
            if time_faces == None:
                final_timestamps[t] = []
            final_timestamps[t].append(f)
        # Check if there is another portion of the response
        try:
            next_token = response['NextToken']
        except:
            break
    # Return the final dictionary
    print('Complete')
    return final_timestamps

In [5]:
%%writefile video_processor.py

import cv2
import numpy as np
from moviepy.editor import *
import os

def anonymize_face_pixelate(frame, face_x, face_w, face_y, face_h, blocks=10):
    image = frame[face_y:face_y+face_h, face_x:face_x+face_w]
    # divide the input image into NxN blocks
    (h, w) = image.shape[:2]
    xSteps = np.linspace(0, w, blocks + 1, dtype="int")
    ySteps = np.linspace(0, h, blocks + 1, dtype="int")

    # loop over the blocks in both the x and y direction
    for i in range(1, len(ySteps)):
        for j in range(1, len(xSteps)):
            # compute the starting and ending (x, y)-coordinates
            # for the current block
            startX = xSteps[j - 1]
            startY = ySteps[i - 1]
            endX = xSteps[j]
            endY = ySteps[i]

            # extract the ROI using NumPy array slicing, compute the
            # mean of the ROI, and then draw a rectangle with the
            # mean RGB values over the ROI in the original image
            roi = image[startY:endY, startX:endX]
            (B, G, R) = [int(x) for x in cv2.mean(roi)[:3]]
            cv2.rectangle(image, (startX, startY), (endX, endY),
                          (B, G, R), -1)

    frame[face_y:face_y+face_h, face_x:face_x+face_w] = image
    return frame

def apply_faces_to_video(timestamps, local_path_to_video, local_output, video_metadata, color=(255,0,0), thickness=2):
    # Extract video info
    frame_rate = video_metadata["FrameRate"]
    frame_height = video_metadata["FrameHeight"]
    frame_width = video_metadata["FrameWidth"]
    # Set up support for OpenCV
    frame_counter = 0
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    # Create the file pointers
    v = cv2.VideoCapture(local_path_to_video)
    out = cv2.VideoWriter(
        filename=local_output, 
        fourcc=fourcc, 
        fps=int(frame_rate), 
        frameSize=(frame_width, frame_height)
    )
    # Open the video
    while v.isOpened():
        # Get frames until available
        has_frame, frame = v.read()
        if has_frame:
            for t in timestamps:
                faces = timestamps.get(t)
                lower_bound = int(int(t)/1000*int(frame_rate))
                upper_bound = int(int(t)/1000*int(frame_rate))+(int(frame_rate)/2)
                if (frame_counter >= lower_bound) and (frame_counter <= upper_bound):
                    for f in faces:
                        x = int(f['Left']*frame_width)
                        y = int(f['Top']*frame_height)
                        w = int(f['Width']*frame_width)
                        h = int(f['Height']*frame_height)
                        #frame = cv2.rectangle(frame, (x,y), (x+w,y+h), color, thickness)
                        frame = anonymize_face_pixelate(frame, x, w, y, h, 50)
            out.write(frame)
            frame_counter += 1
        else:
            break

    out.release()
    v.release()
    cv2.destroyAllWindows()
    print(f"Complete. {frame_counter} frames were written.")

def integrate_audio(original_video, output_video, audio_path='/tmp/audio.mp3'):
    # Extract audio
    my_clip = VideoFileClip(original_video)
    my_clip.audio.write_audiofile(audio_path)

    # Join output video with extracted audio
    videoclip = VideoFileClip(output_video)
    audioclip = AudioFileClip(audio_path)
    new_audioclip = CompositeAudioClip([audioclip])
    videoclip.audio = new_audioclip
    videoclip.write_videofile(output_video)

    # Delete audio
    os.remove(audio_path)

    print('Complete')

Overwriting video_processor.py


In [6]:
%%writefile app.py

import json
import logging
import os
import urllib

import boto3
import botocore
import cv2
import numpy as np

from rekognition import check_format_and_size, start_face_detection, get_timestamps_and_faces
from video_processor import apply_faces_to_video

logger = logging.getLogger()
logger.setLevel(logging.INFO)

reko = boto3.client('rekognition')
s3 = boto3.client('s3')

def lambda_handler(event, context):

    for record in event['Records']:

        # verify event has reference to S3 object
        try:
            # get metadata of file uploaded to Amazon S3
            bucket = record['s3']['bucket']['name']
            key = urllib.parse.unquote_plus(record['s3']['object']['key'])
            size = int(record['s3']['object']['size'])
            filename = key.split('/')[-1]
            local_filename = '/tmp/{}'.format(filename)
        except KeyError:
            error_message = 'Lambda invoked without S3 event data. Event needs to reference a S3 bucket and object key.'
            logger.log(error_message)
            continue

        # verify file and its size
        try:
            assert check_format_and_size(filename, size)
        except:
            error_message = 'Unsupported file type. Amazon Rekognition Video support MOV and MP4 lower than 10 GB in size'
            logger.log(error_message)
            continue

        # download file locally to /tmp retrieve metadata
        try:
            s3.download_file(bucket, key, local_filename)
        except botocore.exceptions.ClientError:
            error_message = 'Lambda role does not have permission to call GetObject for the input S3 bucket, or object does not exist.'
            logger.log(error_message)
            continue

        # use Amazon Rekognition to detect faces in image uploaded to Amazon S3
        try:
            job_id = start_face_detection(bucket, key, 1, reko)
            response = wait_for_completion(job_id, reko_client=reko)
        except rekognition.exceptions.AccessDeniedException:
            error_message = 'Lambda role does not have permission to call DetectFaces in Amazon Rekognition.'
            logger.log(error_message)
            continue
        except rekognition.exceptions.InvalidS3ObjectException:
            error_message = 'Unable to get object metadata from S3. Check object key, region and/or access permissions for input S3 bucket.'
            logger.log(error_message)
            continue

        try:
            timestamps=get_timestamps_and_faces(response, job_id, reko)
            apply_faces_to_video(timestamps, local_path_to_video, local_output, response["VideoMetadata"])
        except Exception as e:
            print(e)
            continue

        # uploaded modified image to Amazon S3 bucket
        try:
            s3.upload_file(local_output, bucket, key)
        except boto3.exceptions.S3UploadFailedError:
            error_message = 'Lambda role does not have permission to call PutObject for the output S3 bucket.'
            add_failed(bucket, error_message, failed_records, key)
            continue

        # clean up /tmp
        if os.path.exists(local_filename):
            os.remove(local_filename)

        successful_records.append({
            "bucket": bucket,
            "key": key
        })

    return {
        "statusCode": 200,
        "body": json.dumps(
            {
                "cv2_version": cv2.__version__,
                "failed_records": failed_records,
                "successful_records": successful_records
            }
        )
    }

Writing app.py


In [8]:
from rekognition import boto3_client, start_face_detection, wait_for_completion, get_timestamps_and_faces
from video_processor import apply_faces_to_video, integrate_audio

bucket = 'datasets-dggallit'
video = 'rekognition-video-demo/video-test.mp4'
local_path_to_video = 'videos/video-test.mp4'
local_output = 'videos/output.mp4'

reko = boto3_client()
job_id = start_face_detection(bucket, video, 1, reko)
response = wait_for_completion(job_id, reko_client=reko)
timestamps=get_timestamps_and_faces(response, job_id, reko)
apply_faces_to_video(timestamps, local_path_to_video, local_output, response["VideoMetadata"])
# integrate_audio('videos/video-test.mp4', 'videos/output.mp4')

..Complete
..Complete
Complete. 2246 frames were written.
chunk:   0%|          | 0/1654 [00:00<?, ?it/s, now=None]MoviePy - Done.
Moviepy - Building video videos/output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp3
t:   1%|          | 17/2247 [00:00<00:13, 162.48it/s, now=None]MoviePy - Done.
Moviepy - Writing video videos/output.mp4

Moviepy - Done !
Moviepy - video ready videos/output.mp4
Complete
